🪙🎯 PLAN ADAPTÉ À TON SUJET : COMMODITIES & FACTORS
0. Executive Summary
•	Brève présentation des objectifs, méthodo, résultats majeurs et recommandations (1 page max).

Annexes
•	Tables de performances détaillées, graphiques, codes, etc.



# Volatility Targeting Strategies for Commodities and Factors

## Project Overview

This notebook implements and analyzes volatility targeting strategies across two asset classes:
1. Traditional investment factors (Market, Size, Value, Momentum)
2. Commodity markets (Gold, Oil, Copper, etc.)

Our goal is to determine whether the volatility targeting approach documented by Moreira & Muir (2017) in equity markets extends effectively to commodity markets, which exhibit different volatility patterns and economic drivers.

### Key Research Questions

1. Does volatility targeting improve risk-adjusted performance in commodity markets?
2. Which volatility estimation methods produce the most effective scaling signals?
3. How do transaction costs and leverage constraints affect strategy implementation?
4. Do volatility-managed commodities provide different benefits across economic cycles?

### Methodology Summary

1. Establish baseline performance for unmanaged factor and commodity portfolios
2. Implement volatility forecasting models (Realized, EWMA, GARCH)
3. Apply the Moreira & Muir scaling framework to both asset classes
4. Analyze performance metrics across different market regimes
5. Test robustness to implementation constraints

1. Introduction
•	Pourquoi les commodities sont pertinentes (diversification, inflation hedge, rôle macro…).
•	Pourquoi tester la stratégie sur à la fois des facteurs (benchmark académique) et des commodities (actifs concrets).
•	Double objectif :
o	Valider la stratégie sur des facteurs (référence théorique).
o	Tester sa pertinence sur un univers plus opérationnel : les matières premières.

2. Théorie des stratégies volatility targeting
•	Rappel du modèle de Moreira & Muir (scaling des rendements).
•	Cas spécifiques aux commodities : volatilité plus élevée, effets saisonniers, impact des roll-overs.
•	Limites potentielles : absence de facteur “fondamental” clair comme pour les actions.

## 1. Moreira & Muir Model for Return Scaling

### 1.1 Theoretical Framework

1. **Core Premise:**
   1. Volatility is persistent and predictable
   2. Returns have little-to-no relationship with future volatility
   3. Risk-adjusted returns are higher in low-volatility periods

2. **Mathematical Foundation:**
   1. Standard portfolio theory suggests inverse relationship between optimal allocation and variance:
      - $w_t = \frac{\mu_t}{\gamma \sigma_t^2}$ where:
      - $w_t$ = portfolio weight at time t
      - $\mu_t$ = expected return
      - $\sigma_t^2$ = expected variance
      - $\gamma$ = risk aversion parameter
   2. Moreira & Muir's key insight: When $\mu_t$ is relatively constant, optimal allocation should be inversely proportional to variance

3. **Implementation Framework:**
   1. Scale exposure to risky assets inversely with predicted volatility
   2. Target constant level of volatility over time
   3. Mathematical expression: $r_{t+1}^{scaled} = \frac{\sigma_{target}}{\hat{\sigma}_t} \times r_{t+1}$

4. **Key Academic Sources:**
   1. Moreira, A., & Muir, T. (2017). Volatility-Managed Portfolios. Journal of Finance, 72(4), 1611-1644
   2. Fleming, J., Kirby, C., & Ostdiek, B. (2001). The Economic Value of Volatility Timing. Journal of Finance, 56(1), 329-352
   3. Harvey, C. R., Hoyle, E., Korgaonkar, R., Rattray, S., Sargaison, M., & Van Hemert, O. (2018). The Impact of Volatility Targeting. Journal of Portfolio Management, 45(1), 14-33


### 1.2 Practical Implementation Steps

1. **Volatility Target Selection:**
   1. Define target annualized volatility (e.g., 10%, 15%, 20%)
   2. Consider risk tolerance, regulatory constraints, and asset class characteristics
   3. For commodities: typically higher targets (15-20%) may be appropriate due to higher native volatility

2. **Volatility Estimation Window:**
   1. Short-term reactivity vs. long-term stability trade-off
   2. Common choices: 1-month (21 days), 3-months (63 days), or 6-months (126 days)
   3. Commodity-specific consideration: account for potential seasonality effects

3. **Position Sizing Algorithm:**
   1. Calculate scaling factor: $s_t = \frac{\sigma_{target}}{\hat{\sigma}_t}$
   2. Apply scaling to position: $Position_t = s_t \times BasePosition_t$
   3. Implement maximum leverage constraints (typically 2x-3x)

4. **Rebalancing Frequency:**
   1. Daily, weekly, or monthly rebalancing based on new volatility estimates
   2. Consider trading costs vs. tracking error to target volatility
   3. Potential use of rebalancing bands to reduce turnover

5. **Potential Implementation Challenges:**
   1. Look-ahead bias risk in backtesting (ensure proper time-indexing)
   2. Transaction costs from frequent rebalancing
   3. Cash management during high volatility periods (when scaling < 1)
   4. Futures-specific issues for commodities (margin requirements, roll costs)


## 2. Volatility Prediction Methods

### 2.1 Theoretical Overview of Volatility Estimation

1. **Realized Volatility:**
   1. Definition: Historical standard deviation of returns over a fixed lookback window
   2. Mathematical expression: $\sigma_t^{realized} = \sqrt{\frac{1}{n-1} \sum_{i=t-n}^{t-1} (r_i - \bar{r})^2}$
   3. Key parameters: Window length (n), sampling frequency
   4. Strengths: Simple to implement, model-free, transparent
   5. Weaknesses: Equal weighting of all observations, slow to adapt to volatility regime changes

2. **Exponentially Weighted Moving Average (EWMA):**
   1. Definition: Weighted average of squared returns with exponentially decaying weights
   2. Mathematical expression: $\sigma_t^2 = (1-\lambda) r_{t-1}^2 + \lambda \sigma_{t-1}^2$
   3. Key parameter: Decay factor λ (typically 0.94 for daily data, per RiskMetrics)
   4. Strengths: More responsive to recent market changes, computationally efficient
   5. Weaknesses: Selection of optimal λ, no mean-reversion component

3. **GARCH Models:**
   1. Definition: Generalized Autoregressive Conditional Heteroskedasticity - models variance as function of past returns and past variance
   2. Mathematical expression (GARCH(1,1)): $\sigma_t^2 = \omega + \alpha r_{t-1}^2 + \beta\sigma_{t-1}^2$
   3. Key parameters: ω (long-term variance), α (return shock impact), β (volatility persistence)
   4. Strengths: Mean-reverting, captures volatility clustering, models persistence
   5. Weaknesses: Parameter estimation complexity, assumption of normally distributed returns

4. **Academic Sources:**
   1. Hansen, P. R., & Lunde, A. (2005). A forecast comparison of volatility models: Does anything beat a GARCH(1,1)?. Journal of Applied Econometrics, 20(7), 873-889
   2. Andersen, T. G., Bollerslev, T., Diebold, F. X., & Labys, P. (2003). Modeling and forecasting realized volatility. Econometrica, 71(2), 579-625
   3. Poon, S. H., & Granger, C. W. (2003). Forecasting volatility in financial markets: A review. Journal of Economic Literature, 41(2), 478-539

### 2.2 Implementation Methodology

1. **Data Preparation for Volatility Estimation:**
   1. Convert price series to return series (log or simple returns)
   2. Screen for outliers that might distort volatility estimates
   3. Handle missing data points (interpolation or omission)
   4. Consider the appropriate return frequency (daily vs. intraday)

2. **Realized Volatility Implementation Steps:**
   1. Define the lookback window (e.g., 21, 63, or 126 days)
   2. Calculate rolling standard deviation of returns
   3. Annualize the estimate by multiplying by $\sqrt{252}$ (for daily data)
   4. Consider window length trade-offs: shorter windows are more reactive but noisier
   5. Potential improvement: Use overlapping windows to increase estimation efficiency

3. **EWMA Implementation Steps:**
   1. Select decay parameter λ (standard is 0.94, but can be optimized)
   2. Initialize variance estimate (often with realized variance of first window)
   3. Recursively update the estimate with new return observations
   4. Consider adaptive λ approaches for different market regimes
   5. Evaluate half-life of information decay to ensure relevance

4. **GARCH Implementation Steps:**
   1. Select GARCH model specification (typically GARCH(1,1) is sufficient)
   2. Estimate parameters using maximum likelihood (requires optimization)
   3. Perform model diagnostics (standardized residuals, information criteria)
   4. Generate out-of-sample forecasts for the next period
   5. Consider extensions for asymmetry (EGARCH, GJR-GARCH) or long memory (FIGARCH)

5. **Implementation Challenges Specific to Commodities:**
   1. Seasonality effects requiring seasonal adjustment
   2. Discontinuities around futures roll dates
   3. Fat-tailed return distributions more common than in equities
   4. Structural breaks due to supply/demand shocks
   5. Market-specific factors (e.g., weather impacts for agricultural commodities)

In [ ]:
# Implementation of key volatility estimation methods
import numpy as np
import pandas as pd
from arch import arch_model

def calculate_volatility_measures(returns, realized_window=21, ewma_lambda=0.94, garch_p=1, garch_q=1):
    """
    Calculate different volatility estimates for a return series
    
    Parameters:
    -----------
    returns : pandas.Series
        Daily returns series
    realized_window : int
        Lookback window for realized volatility
    ewma_lambda : float
        Decay factor for EWMA volatility (between 0 and 1)
    garch_p : int
        GARCH lag order
    garch_q : int
        ARCH lag order
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame with different volatility estimates
    """
    # Calculate annualization factor
    ann_factor = np.sqrt(252)  # Assuming daily returns
    
    # 1. Realized Volatility
    realized_vol = returns.rolling(window=realized_window).std() * ann_factor
    
    # 2. EWMA Volatility
    ewma_vol = pd.Series(index=returns.index)
    ewma_vol.iloc[0] = returns.iloc[0] if not np.isnan(returns.iloc[0]) else 0
    
    # Initialize with realized vol of first window if possible
    if len(returns) >= realized_window:
        init_var = returns.iloc[:realized_window].var()
    else:
        init_var = returns.iloc[0]**2
    
    # Recursive calculation
    var = init_var
    for t in range(1, len(returns)):
        if np.isnan(returns.iloc[t-1]):
            var = var  # Keep previous variance if return is missing
        else:
            var = (1 - ewma_lambda) * returns.iloc[t-1]**2 + ewma_lambda * var
        ewma_vol.iloc[t] = np.sqrt(var) * ann_factor
    
    # 3. GARCH Volatility (using arch_model from arch package)
    # This requires a longer time series for reliable estimation
    garch_vol = pd.Series(index=returns.index)
    
    # Only attempt GARCH if we have sufficient data
    min_obs = 100  # Minimum observations for GARCH estimation
    if len(returns.dropna()) > min_obs:
        try:
            # Fit GARCH model
            model = arch_model(returns.dropna(), vol='Garch', p=garch_p, q=garch_q)
            model_fit = model.fit(disp='off')
            
            # Extract conditional volatility
            garch_conditional_vol = model_fit.conditional_volatility
            
            # Map back to original index
            garch_vol = pd.Series(
                garch_conditional_vol * ann_factor,
                index=returns.dropna().index
            ).reindex(returns.index)
        except:
            print("GARCH estimation failed. Using NaN values.")
    
    # Combine all volatility measures
    vol_df = pd.DataFrame({
        'Realized_Vol': realized_vol,
        'EWMA_Vol': ewma_vol,
        'GARCH_Vol': garch_vol
    })
    
    return vol_df

# Function to plot volatility comparison
def plot_volatility_comparison(vol_df, title="Volatility Estimation Comparison", figsize=(12, 6)):
    plt.figure(figsize=figsize)
    
    for col in vol_df.columns:
        plt.plot(vol_df.index, vol_df[col], label=col)
    
    plt.title(title)
    plt.xlabel('Date')
    plt.ylabel('Annualized Volatility')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    
    return plt

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')

# Example function implementing the Moreira & Muir scaling approach
def volatility_scaling(returns, vol_window=21, target_vol=0.15, max_leverage=3.0, ann_factor=252):
    """
    Implement volatility targeting according to Moreira & Muir (2017)
    
    Parameters:
    -----------
    returns: pandas.Series
        Daily returns of the asset
    vol_window: int
        Lookback window for volatility estimation (in trading days)
    target_vol: float
        Target annualized volatility (as decimal, e.g., 0.15 for 15%)
    max_leverage: float
        Maximum allowed leverage
    ann_factor: int
        Annualization factor (252 for daily returns)
        
    Returns:
    --------
    tuple: (scaled_returns, scaling_factors)
        scaled_returns: pandas.Series - volatility-scaled returns
        scaling_factors: pandas.Series - leverage applied at each point
    """
    # Calculate rolling volatility (standard deviation)
    rolling_vol = returns.rolling(window=vol_window).std() * np.sqrt(ann_factor)
    
    # Calculate scaling factors (with max leverage constraint)
    scaling_factors = target_vol / rolling_vol
    scaling_factors = scaling_factors.clip(upper=max_leverage)
    
    # Align scaling factors with next-day returns (avoid look-ahead bias)
    aligned_scaling = scaling_factors.shift(1)
    
    # Apply scaling to returns
    scaled_returns = returns * aligned_scaling
    
    # Remove NaN values from the start of the series
    valid_index = ~scaled_returns.isna()
    scaled_returns = scaled_returns[valid_index]
    scaling_used = aligned_scaling[valid_index]
    
    return scaled_returns, scaling_used

## 3. Rolling Strategy Construction

### 3.1 Theoretical Framework for Rolling Implementation

1. **Rolling Window Approach in Financial Modeling:**
   1. Definition: A method that uses overlapping or non-overlapping windows of historical data to estimate parameters and make forward-looking decisions
   2. Purpose in volatility targeting: Ensures realistic implementation by using only information available at each decision point
   3. Common window parameters: Estimation window (lookback period) and holding period (rebalancing frequency)
   4. Mathematical representation: For each time point t, use data from [t-n, t-1] to inform decision at time t

2. **Out-of-Sample Implementation Requirements:**
   1. Strict time-indexing to prevent look-ahead bias (information from time t cannot inform decisions at t)
   2. Parameter re-estimation at each step rather than fixed parameters for the entire backtest
   3. Walk-forward validation approach to simulate real-world implementation
   4. Consideration of data availability constraints at each historical point

3. **Rolling Strategy Calibration:**
   1. Parameter stability assessment across different market regimes
   2. Sensitivity analysis to window length selection
   3. Regime-dependent parameter adaptation methods
   4. Statistical tests for parameter significance and robustness over time

4. **Academic Sources:**
   1. Pesaran, M. H., & Timmermann, A. (2007). Selection of estimation window in the presence of breaks. Journal of Econometrics, 137(1), 134-161
   2. Inoue, A., Jin, L., & Rossi, B. (2017). Rolling window selection for out-of-sample forecasting with time-varying parameters. Journal of Econometrics, 196(1), 55-67
   3. Barroso, P., & Santa-Clara, P. (2015). Momentum has its moments. Journal of Financial Economics, 116(1), 111-120

### 3.2 Implementation Methodology

1. **Setting Up the Rolling Framework:**
   1. Define estimation window length for volatility calculation (e.g., 63 days for 3-month window)
   2. Define rebalancing frequency (e.g., daily, weekly, or monthly)
   3. Establish starting point with sufficient historical data (minimum estimation window)
   4. Create data structures to store results (scaled returns, positions, scaling factors)
   5. Implement chronological iteration through time series to simulate real-world implementation

2. **Volatility Forecast Step:**
   1. For each time point t, use returns from [t-n, t-1] to estimate volatility
   2. Apply your selected volatility models (realized, EWMA, GARCH)
   3. Store volatility forecasts in forward-looking data structure
   4. Compare volatility forecasts across models to assess stability
   5. Consider conditional forecasts based on market regime indicators

3. **Position Sizing Step:**
   1. Calculate scaling factor for each asset as σ_target/σ_forecast
   2. Apply maximum leverage constraints (typically 2x-3x)
   3. Optionally implement minimum position sizes for practical considerations
   4. Create position matrix through time for each asset and model
   5. Calculate capital allocation required for implementation

4. **Portfolio Rebalancing Step:**
   1. Calculate target position changes based on new scaling factors
   2. Estimate transaction costs from position changes
   3. Implement rebalancing threshold rules to reduce turnover
   4. Calculate net realized returns after accounting for costs
   5. Update portfolio values and weights

5. **Implementation Challenges and Solutions:**
   1. Challenge: Non-synchronous trading hours between commodities and equity factors
      - Solution: Align data using previous day closing prices for next day signals
   2. Challenge: Data gaps in commodity futures during roll periods
      - Solution: Implement roll-adjusted continuous price series
   3. Challenge: Computational efficiency in daily rebalancing
      - Solution: Vectorized operations and optimized calculation methods
   4. Challenge: Parameter instability during extreme volatility
      - Solution: Implement regime-switching models or shrinkage estimators
   5. Challenge: Trading limitations during market stress
      - Solution: Incorporate liquidity-based position caps during high volatility

In [ ]:
# Implementation of rolling strategy construction
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

def construct_rolling_strategy(returns_df, vol_estimation_functions, 
                               estimation_window=63, target_vol=0.15, 
                               max_leverage=3.0, rebalance_freq='D',
                               transaction_cost_bps=10):
    """
    Implements rolling volatility-targeting strategy across multiple assets
    
    Parameters:
    -----------
    returns_df : pandas.DataFrame
        DataFrame with asset returns in columns
    vol_estimation_functions : dict
        Dictionary mapping vol model names to their functions
    estimation_window : int
        Lookback window for volatility estimation in days
    target_vol : float
        Target annualized volatility (decimal)
    max_leverage : float
        Maximum leverage constraint
    rebalance_freq : str
        Pandas frequency string ('D'=daily, 'W'=weekly, 'M'=monthly)
    transaction_cost_bps : float
        Transaction costs in basis points
        
    Returns:
    --------
    dict of pandas.DataFrame
        Dict containing scaled returns, scaling factors, and positions
    """
    # Initialize results containers
    assets = returns_df.columns
    models = list(vol_estimation_functions.keys())
    
    # Create rebalancing dates (daily, weekly, monthly)
    all_dates = returns_df.index
    rebalance_dates = returns_df.asfreq(rebalance_freq).index
    
    # Store results in multi-level DataFrames
    scaled_returns = {}
    scaling_factors = {}
    positions = {}
    turnover = {}
    
    # Annualization factor
    ann_factor = np.sqrt(252)  # Assuming daily returns
    
    # Loop through volatility models
    for model_name, vol_func in vol_estimation_functions.items():
        print(f"Processing {model_name} model...")
        
        # Initialize model-specific results
        scaled_returns[model_name] = pd.DataFrame(index=all_dates, columns=assets)
        scaling_factors[model_name] = pd.DataFrame(index=all_dates, columns=assets)
        positions[model_name] = pd.DataFrame(index=all_dates, columns=assets)
        turnover[model_name] = pd.DataFrame(index=all_dates, columns=assets)
        
        # Loop through assets
        for asset in tqdm(assets, desc=f"Assets for {model_name}"):
            asset_returns = returns_df[asset]
            
            # Initialize positions and previous positions
            prev_position = 0
            
            # Implement rolling strategy
            for t in range(estimation_window, len(all_dates)):
                current_date = all_dates[t]
                
                # Only rebalance on rebalance dates
                if current_date in rebalance_dates:
                    # Get historical data for estimation window
                    hist_returns = asset_returns.iloc[t-estimation_window:t]
                    
                    # Calculate volatility forecast using the specified model
                    vol_forecast = vol_func(hist_returns)
                    
                    # Calculate scaling factor with leverage constraint
                    scaling = min(target_vol / (vol_forecast * ann_factor), max_leverage)
                    
                    # Update position
                    new_position = scaling
                    
                    # Calculate turnover
                    trade = new_position - prev_position
                    turnover[model_name].loc[current_date, asset] = abs(trade)
                    
                    # Apply transaction costs
                    tc = abs(trade) * (transaction_cost_bps / 10000)  # Convert bps to decimal
                    
                    # Store results
                    scaling_factors[model_name].loc[current_date, asset] = scaling
                    positions[model_name].loc[current_date, asset] = new_position
                    
                    # Update previous position for next iteration
                    prev_position = new_position
                else:
                    # Keep previous position on non-rebalancing days
                    positions[model_name].loc[current_date, asset] = prev_position
                    scaling_factors[model_name].loc[current_date, asset] = prev_position
                    turnover[model_name].loc[current_date, asset] = 0
                
                # Calculate scaled return for the day
                if t < len(all_dates) - 1:  # Ensure we're not at the last data point
                    next_return = asset_returns.iloc[t+1]
                    position = positions[model_name].loc[current_date, asset]
                    
                    # Apply scaling to return
                    scaled_ret = next_return * position
                    
                    # Store scaled return (properly aligned to next day)
                    next_date = all_dates[t+1]
                    scaled_returns[model_name].loc[next_date, asset] = scaled_ret
        
        # Forward fill positions and scaling factors for continuity
        positions[model_name] = positions[model_name].ffill()
        scaling_factors[model_name] = scaling_factors[model_name].ffill()
    
    return {
        "scaled_returns": scaled_returns,
        "scaling_factors": scaling_factors,
        "positions": positions,
        "turnover": turnover
    }

# Example usage
# Define volatility estimation functions dictionary
def realized_volatility(returns, window=21):
    """Simple realized volatility estimation"""
    return returns.std()

def ewma_volatility(returns, lambda_param=0.94):
    """EWMA volatility estimation"""
    # Initialize with realized vol
    var = returns.var()
    # Recursively calculate EWMA
    for ret in returns:
        if not np.isnan(ret):
            var = (1 - lambda_param) * ret**2 + lambda_param * var
    return np.sqrt(var)

# Define vol_estimation_functions dictionary
vol_models = {
    "Realized_21d": lambda x: realized_volatility(x, window=21),
    "Realized_63d": lambda x: realized_volatility(x, window=63),
    "EWMA_0.94": lambda x: ewma_volatility(x, lambda_param=0.94)
}

 
3. Données
•	Commodities :
o	Sélection de 5 à 10 actifs (Gold, Oil, Copper, Corn, Natural Gas, etc.).
o	Source : Bloomberg, Yahoo Finance, ou autres.
o	Fréquence : journalière ou mensuelle selon la dispo.
•	Facteurs (Fama-French + Mom) :
o	Données européennes si possible, sinon US.
o	Source : site de Kenneth French.
•	Nettoyage, synchronisation, traitement des jours fériés.

4. Construction des stratégies
•	Formule de scaling avec σ_target / σ_pred.
•	Volatilité prédite via 3 approches :
o	Réalisée 1 mois
o	Réalisée 6 mois
o	EWMA ou GARCH (ou une approche machine learning simple si tu veux te démarquer).
•	Mise en œuvre rolling pour éviter look-ahead bias.
•	Comparaison entre stratégie naïve et stratégie scalée.

 
5. Analyse des performances
•	Par univers : facteurs vs commodities.
•	Statistiques : rendement, volatilité, Sharpe, max drawdown, skewness, kurtosis, expected shortfall, turnover.
•	Régressions alpha & appraisal ratio (si tu as un benchmark).
•	Résultats in-sample vs out-of-sample.
•	Focus sur les périodes de crise (2008, 2020, etc.).

6. Analyse Cycle Économique (optionnelle pour commodities mais différenciante)
•	Utilisation des dates NBER.
•	Performances en expansion vs récession.
•	Peut révéler des comportements défensifs / cycliques des commodities.

7. Contraintes réelles : transaction costs et leverage
•	Commodities : effets spécifiques (roll-over, coûts implicites).
•	Facteurs : turnover élevé sur momentum par exemple.
•	Scénarios avec 10, 30, 50 bps.
•	Propositions :
o	Seuils de rebalancement.
o	Intégration des coûts dans la fonction d’optimisation.
•	Étude de sensibilité aux contraintes de levier.

8. Synthèse et recommandations
•	Résumé des résultats clés sur les deux univers.
•	Intérêt concret du volatility targeting sur des actifs réels.
•	Limites pratiques (data quality, implémentation).
•	Recommandations pour une mise en œuvre future.
 

## 4. Comparative Performance Analysis

### 4.1 Theoretical Framework for Performance Evaluation

1. **Performance Measurement in Risk-Adjusted Context:**
   1. Definition: The assessment of investment returns in relation to the risk taken to achieve those returns
   2. Core principle: Higher returns alone are insufficient; they must be evaluated per unit of risk
   3. Two primary dimensions: absolute performance and risk-adjusted metrics
   4. Mathematical foundation: Mean-variance framework and its extensions

2. **Key Performance Metrics:**
   1. **Return Metrics:**
      1. Annualized return: $R_{ann} = (1 + R_{p})^{\frac{252}{T}} - 1$ (for daily returns)
      2. Cumulative return: $R_{cum} = \prod_{t=1}^{T} (1 + r_t) - 1$
   2. **Risk Metrics:**
      1. Volatility: $\sigma = \sqrt{\frac{252}{T-1}\sum_{t=1}^{T}(r_t - \bar{r})^2}$
      2. Maximum drawdown: $MDD = \max_{\tau \in (0,t)} [\frac{V(\tau) - V(t)}{V(\tau)}]$
      3. Value-at-Risk (VaR): $P(r_t < VaR_\alpha) = \alpha$
      4. Expected Shortfall: $ES_\alpha = E[r_t | r_t < VaR_\alpha]$
   3. **Risk-Adjusted Metrics:**
      1. Sharpe ratio: $SR = \frac{R_p - R_f}{\sigma_p}$
      2. Sortino ratio: $SOR = \frac{R_p - R_f}{\sigma_{down}}$
      3. Calmar ratio: $CR = \frac{R_{ann}}{MDD}$
   4. **Trading Implementation Metrics:**
      1. Turnover: $TO = \frac{1}{T}\sum_{t=1}^T \sum_{i=1}^N |w_{i,t} - w_{i,t-1}|$
      2. Average leverage: $Lev_{avg} = \frac{1}{T}\sum_{t=1}^T \sum_{i=1}^N |w_{i,t}|$
      3. Transaction cost impact: $TC = \sum_{t=1}^T \sum_{i=1}^N |w_{i,t} - w_{i,t-1}| \cdot c$

3. **Statistical Significance Tests:**
   1. Jobson-Korkie test for Sharpe ratio differences
   2. Bootstrap methods for robust confidence intervals
   3. Ledoit-Wolf shrinkage for improved covariance estimation
   4. Multiple hypothesis testing correction (e.g., Bonferroni, False Discovery Rate)

4. **Academic Sources:**
   1. Jobson, J.D., & Korkie, B.M. (1981). Performance hypothesis testing with the Sharpe and Treynor measures. Journal of Finance, 36(4), 889-908
   2. Ledoit, O., & Wolf, M. (2008). Robust performance hypothesis testing with the Sharpe ratio. Journal of Empirical Finance, 15(5), 850-859
   3. Harvey, C.R., & Liu, Y. (2015). Backtesting. Journal of Portfolio Management, 42(1), 13-28
   4. Moreira, A., & Muir, T. (2019). Should long-term investors time volatility? Journal of Financial Economics, 131(3), 507-527

### 4.2 Implementation Methodology

1. **Performance Metric Calculation Framework:**
   1. Define standard evaluation periods (full sample, sub-periods, crisis vs. normal)
   2. Implement consistent periodicity for comparison (daily, monthly, annualized)
   3. Develop a comprehensive performance dashboard for strategy comparison
   4. Establish benchmarks for relative performance assessment
   5. Document all assumptions and methodological choices for transparency

2. **Factor vs. Commodity Strategy Analysis Steps:**
   1. Compare baseline (unmanaged) performance across asset classes
   2. Evaluate incremental benefit of volatility targeting within each asset class
   3. Assess volatility reduction efficiency across different instruments
   4. Examine tail risk mitigation effects during market stress periods
   5. Analyze correlation dynamics between managed and unmanaged strategies

3. **Volatility Model Comparison Methodology:**
   1. Calculate performance metrics for each volatility estimation approach
   2. Implement statistical tests for significant differences between models
   3. Evaluate model-specific performance across different market regimes
   4. Analyze trade-off between model complexity and performance improvement
   5. Assess parameter stability and robustness across asset classes

4. **Transaction Cost Analysis Framework:**
   1. Implement realistic transaction cost models (fixed, proportional, market impact)
   2. Compare turnover profiles across volatility estimation methods
   3. Develop cost-adjusted performance metrics for all strategies
   4. Test sensitivity to cost assumptions (10, 30, 50 bps scenarios)
   5. Evaluate implementation efficiency measures (cost per unit of risk reduction)

5. **Implementation Challenges and Solutions:**
   1. Challenge: Multiple comparison bias in strategy evaluation
      - Solution: Apply multiple hypothesis testing corrections
   2. Challenge: Parameter sensitivity affecting robust conclusions
      - Solution: Implement parameter sensitivity analysis and cross-validation
   3. Challenge: Outlier returns distorting performance metrics
      - Solution: Use robust statistical methods and winsorization
   4. Challenge: Strategy-specific data snooping
      - Solution: Implement out-of-sample testing and walk-forward validation
   5. Challenge: Time-varying performance dynamics
      - Solution: Analyze rolling-window performance metrics and regime-dependent analysis

In [ ]:
# Implementation of performance analysis functions
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

def calculate_performance_metrics(returns, risk_free=0.0, periods_per_year=252):
    """
    Calculate comprehensive performance metrics for a returns series
    
    Parameters:
    -----------
    returns : pandas.Series or pandas.DataFrame
        Series or DataFrame of returns
    risk_free : float or pandas.Series
        Risk-free rate (constant or time series)
    periods_per_year : int
        Number of periods in a year (252 for daily, 12 for monthly)
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame with performance metrics for each series
    """
    # Convert Series to DataFrame if needed
    if isinstance(returns, pd.Series):
        returns = pd.DataFrame(returns)
    
    # Initialize results dictionary
    metrics = {}
    
    for col in returns.columns:
        series = returns[col].dropna()
        if len(series) == 0:
            continue
            
        # Calculate return metrics
        cum_return = (1 + series).prod() - 1
        ann_return = (1 + cum_return) ** (periods_per_year / len(series)) - 1
        
        # Calculate risk metrics
        volatility = series.std() * np.sqrt(periods_per_year)
        neg_returns = series[series < 0]
        downside_vol = neg_returns.std() * np.sqrt(periods_per_year) if len(neg_returns) > 0 else 0
        
        # Calculate drawdowns
        wealth_index = (1 + series).cumprod()
        previous_peaks = wealth_index.cummax()
        drawdowns = (wealth_index - previous_peaks) / previous_peaks
        max_drawdown = drawdowns.min()
        
        # Calculate risk-adjusted metrics
        sharpe = (ann_return - risk_free) / volatility if volatility != 0 else np.nan
        sortino = (ann_return - risk_free) / downside_vol if downside_vol != 0 else np.nan
        calmar = ann_return / abs(max_drawdown) if max_drawdown != 0 else np.nan
        
        # Calculate additional statistics
        skewness = series.skew()
        kurtosis = series.kurtosis()  # Excess kurtosis
        
        # Calculate VaR and Expected Shortfall (CVaR)
        var_95 = np.percentile(series, 5)  # 95% VaR (5th percentile of returns)
        es_95 = series[series <= var_95].mean() if len(series[series <= var_95]) > 0 else var_95
        
        # Store all metrics
        metrics[col] = {
            'Cumulative Return': cum_return,
            'Annualized Return': ann_return,
            'Annualized Volatility': volatility,
            'Sharpe Ratio': sharpe,
            'Sortino Ratio': sortino,
            'Max Drawdown': max_drawdown,
            'Calmar Ratio': calmar,
            'Skewness': skewness,
            'Kurtosis': kurtosis,
            'VaR (95%)': var_95,
            'Expected Shortfall (95%)': es_95,
            'Positive Months (%)': len(series[series > 0]) / len(series) * 100,
            'Observation Count': len(series)
        }
    
    # Convert to DataFrame
    metrics_df = pd.DataFrame(metrics)
    
    return metrics_df

In [ ]:
def plot_strategy_comparison(returns_dict, figsize=(12, 8), title="Strategy Comparison"):
    """
    Plot cumulative returns and drawdowns for multiple strategies
    
    Parameters:
    -----------
    returns_dict : dict
        Dictionary mapping strategy names to return Series
    figsize : tuple
        Figure size
    title : str
        Plot title
        
    Returns:
    --------
    fig : matplotlib.figure.Figure
        Figure object with plots
    """
    fig, axes = plt.subplots(2, 1, figsize=figsize, gridspec_kw={'height_ratios': [2, 1]})
    
    # Plot cumulative returns
    for name, returns in returns_dict.items():
        cum_returns = (1 + returns).cumprod()
        axes[0].plot(cum_returns.index, cum_returns, label=name)
    
    axes[0].set_title(f"{title} - Cumulative Returns")
    axes[0].set_ylabel("Cumulative Return")
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Plot drawdowns
    for name, returns in returns_dict.items():
        wealth_index = (1 + returns).cumprod()
        previous_peaks = wealth_index.cummax()
        drawdowns = (wealth_index - previous_peaks) / previous_peaks
        axes[1].plot(drawdowns.index, drawdowns, label=name)
    
    axes[1].set_title("Drawdowns")
    axes[1].set_ylabel("Drawdown")
    axes[1].set_ylim(bottom=-1, top=0.05)  # Adjust as needed
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig

def analyze_market_regimes(returns, regime_data, regime_column='Regime'):
    """
    Analyze performance across different market regimes
    
    Parameters:
    -----------
    returns : pandas.DataFrame
        DataFrame with strategy returns as columns
    regime_data : pandas.DataFrame
        DataFrame with regime classification
    regime_column : str
        Column name in regime_data that contains regime information
        
    Returns:
    --------
    dict of pandas.DataFrame
        Dict mapping regime names to performance metrics
    """
    # Merge returns with regime data
    merged_data = returns.join(regime_data[regime_column], how='inner')
    
    # Get unique regimes
    regimes = merged_data[regime_column].unique()
    
    # Calculate performance metrics for each regime
    regime_performance = {}
    
    for regime in regimes:
        # Filter returns for this regime
        regime_returns = merged_data[merged_data[regime_column] == regime].drop(columns=[regime_column])
        
        # Calculate performance metrics
        metrics = calculate_performance_metrics(regime_returns)
        
        regime_performance[regime] = metrics
    
    return regime_performance

# Example usage (would be implemented with real data)
# performance_metrics = calculate_performance_metrics(all_strategy_returns)
# fig = plot_strategy_comparison({'Unmanaged': unmanaged_returns, 'Vol-Managed': vol_managed_returns})
# regime_performance = analyze_market_regimes(all_returns, economic_regimes, regime_column='NBER_Recession')

## 5. Economic Cycle Analysis

### 5.1 Theoretical Framework for Economic Regime Analysis

1. **Economic Cycles and Asset Returns Relationship:**
   1. Definition: The systematic variation in asset returns across different phases of the business cycle
   2. Core premise: Different asset classes exhibit varying sensitivities to economic growth and inflation
   3. Traditional factor model: Returns are decomposed into exposures to economic states
   4. Mathematical representation: $R_{i,t} = \alpha_i + \sum_{j=1}^{K} \beta_{i,j} F_{j,t} + \epsilon_{i,t}$ where $F_{j,t}$ includes economic state variables

2. **Business Cycle Classification Methods:**
   1. **Official Dating Approaches:**
      1. NBER recession dating (U.S. focused, comprehensive but lagged)
      2. CEPR cycle dating (European equivalent of NBER)
      3. OECD CLI (Composite Leading Indicators for international coverage)
   2. **Quantitative Classification Methods:**
      1. GDP growth rate thresholds (e.g., below 0% for recession)
      2. Markov-switching models (Hamilton regime switching approach)
      3. Statistical filtering techniques (Hodrick-Prescott, Kalman)
   3. **Market-Based Indicators:**
      1. Yield curve slopes (term spread as recession predictor)
      2. Credit spreads (widening indicates economic stress)
      3. Volatility regime clustering (GARCH-based regime identification)

3. **Commodity-Specific Economic Sensitivity:**
   1. Pro-cyclical commodities (industrial metals, energy) with positive GDP growth correlation
   2. Counter-cyclical commodities (gold, sometimes agricultural) as inflation/recession hedges
   3. Supply constraint effects creating idiosyncratic cycle responses
   4. Monetary policy transmission effects on commodity pricing

4. **Academic Sources:**
   1. Fama, E. F., & French, K. R. (1989). Business conditions and expected returns on stocks and bonds. Journal of Financial Economics, 25(1), 23-49
   2. Hamilton, J. D. (1989). A new approach to the economic analysis of nonstationary time series and the business cycle. Econometrica, 57(2), 357-384
   3. Gorton, G., & Rouwenhorst, K. G. (2006). Facts and fantasies about commodity futures. Financial Analysts Journal, 62(2), 47-68
   4. Erb, C. B., & Harvey, C. R. (2006). The strategic and tactical value of commodity futures. Financial Analysts Journal, 62(2), 69-97

### 5.2 Implementation Methodology

1. **Constructing the Economic Regime Database:**
   1. Download NBER recession indicators from FRED database (US) or comparable source
   2. Create a binary recession indicator series aligned with asset return dates
   3. Consider additional granularity beyond binary classification (expansion, late-cycle, early recession, late recession)
   4. Validate official dates with real-time indicators to address publication lag issues
   5. Extend regime classification internationally for global commodity analysis

2. **Regime-Conditional Performance Analysis Steps:**
   1. Segment return series by economic regime classification
   2. Calculate conditional performance metrics for each regime
   3. Test statistical significance of performance differentials across regimes
   4. Analyze strategy correlation matrices conditional on economic state
   5. Examine volatility persistence characteristics across different regimes

3. **Volatility Targeting Strategy Adaptation to Economic Regimes:**
   1. Evaluate efficacy of volatility scaling across different economic environments
   2. Compare volatility clustering behavior in expansions versus recessions
   3. Test regime-dependent parameter optimization (adaptive target volatility)
   4. Assess timing delay impact on economic transition periods
   5. Implement regime-switching volatility models to capture structural changes

4. **Factor vs. Commodity Analysis Across Economic States:**
   1. Compare defensive characteristics of volatility-managed portfolios across asset classes
   2. Analyze beta to market conditional on economic regimes
   3. Examine drawdown protection efficacy during recession periods
   4. Test inflation sensitivity differences between managed and unmanaged portfolios
   5. Evaluate diversification benefits across economic environments

5. **Implementation Challenges and Solutions:**
   1. Challenge: Regime classification is only known with certainty in hindsight
      - Solution: Implement probabilistic regime models with leading indicators
   2. Challenge: Limited recession observations in sample period
      - Solution: Synthetic stress testing and bootstrap resampling methods
   3. Challenge: Heterogeneity of recessions (financial, pandemic, supply-shock)
      - Solution: Sub-classify recession types for more granular analysis
   4. Challenge: International economic cycle asynchronicity
      - Solution: Country-specific regimes or global factor extraction
   5. Challenge: Structural breaks in economic relationships
      - Solution: Time-varying parameter models or rolling estimation windows

In [ ]:
# Implementation of economic cycle analysis functions
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_datareader.data import DataReader
from datetime import datetime

def download_nber_recession_data(start_date='1990-01-01', end_date=None):
    """
    Download NBER recession indicators from FRED
    
    Parameters:
    -----------
    start_date : str
        Start date in 'YYYY-MM-DD' format
    end_date : str
        End date in 'YYYY-MM-DD' format (defaults to current date)
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame with recession indicators (1=recession, 0=expansion)
    """
    if end_date is None:
        end_date = datetime.now().strftime('%Y-%m-%d')
    
    try:
        # Download NBER recession indicator data from FRED
        # Note: This requires internet connection and pandas_datareader
        recession_data = DataReader('USREC', 'fred', start_date, end_date)
        recession_data.columns = ['Recession']
        return recession_data
    except Exception as e:
        print(f"Error downloading recession data: {e}")
        print("Creating mock recession data for demonstration purposes...")
        
        # Create mock recession data for demonstration
        date_range = pd.date_range(start=start_date, end=end_date, freq='M')
        mock_data = pd.DataFrame(index=date_range, columns=['Recession'])
        
        # Define some mock recessions (2001, 2008-2009, 2020)
        mock_data['Recession'] = 0
        mock_data.loc['2001-03-01':'2001-11-01', 'Recession'] = 1  # 2001 recession
        mock_data.loc['2007-12-01':'2009-06-01', 'Recession'] = 1  # Global Financial Crisis
        mock_data.loc['2020-02-01':'2020-04-01', 'Recession'] = 1  # COVID-19 initial shock
        
        return mock_data

def align_recession_data_with_returns(returns, recession_data):
    """
    Align recession indicators with return data dates
    
    Parameters:
    -----------
    returns : pandas.DataFrame
        DataFrame with asset returns
    recession_data : pandas.DataFrame
        DataFrame with recession indicators
        
    Returns:
    --------
    pandas.DataFrame
        Returns DataFrame with added recession indicator
    """
    # Ensure recession data has daily frequency if returns are daily
    if recession_data.index.freq != returns.index.freq:
        recession_daily = recession_data.asfreq('D', method='ffill')
    else:
        recession_daily = recession_data
    
    # Align with returns index
    aligned_recession = recession_daily.reindex(returns.index, method='ffill')
    
    # Add to returns DataFrame
    returns_with_regime = returns.copy()
    returns_with_regime['Recession'] = aligned_recession['Recession']
    
    return returns_with_regime

def analyze_regime_performance(returns, regime_column='Recession'):
    """
    Analyze performance metrics conditional on economic regime
    
    Parameters:
    -----------
    returns : pandas.DataFrame
        Returns with regime indicator column
    regime_column : str
        Name of the column containing regime indicator
        
    Returns:
    --------
    dict of pandas.DataFrame
        Performance metrics by regime
    """
    # Get unique regimes
    regimes = returns[regime_column].unique()
    
    performance_by_regime = {}
    asset_columns = [col for col in returns.columns if col != regime_column]
    
    for regime in regimes:
        # Filter returns for current regime
        regime_returns = returns[returns[regime_column] == regime][asset_columns]
        
        # Calculate performance metrics
        metrics = calculate_performance_metrics(regime_returns)
        performance_by_regime[int(regime)] = metrics
    
    return performance_by_regime

def plot_regime_performance_comparison(performance_by_regime, metric='Annualized Return', figsize=(12, 6)):
    """
    Plot performance comparison across economic regimes
    
    Parameters:
    -----------
    performance_by_regime : dict
        Dict of DataFrames with performance metrics by regime
    metric : str
        Performance metric to compare
    figsize : tuple
        Figure size
        
    Returns:
    --------
    matplotlib.figure.Figure
        Figure with regime comparison plot
    """
    # Extract the specified metric from each regime's performance data
    regime_names = {0: 'Expansion', 1: 'Recession'}
    metric_by_regime = {regime_names[regime]: df.loc[metric] for regime, df in performance_by_regime.items()}
    
    # Create DataFrame for plotting
    plot_data = pd.DataFrame(metric_by_regime)
    
    # Create plot
    fig, ax = plt.subplots(figsize=figsize)
    plot_data.plot(kind='bar', ax=ax)
    
    ax.set_title(f'{metric} by Economic Regime')
    ax.set_ylabel(metric)
    ax.set_xlabel('Strategy')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig

# Example usage (would be implemented with actual data)
# recession_data = download_nber_recession_data('2000-01-01')
# returns_with_regime = align_recession_data_with_returns(all_strategy_returns, recession_data)
# regime_performance = analyze_regime_performance(returns_with_regime)
# fig = plot_regime_performance_comparison(regime_performance, metric='Sharpe Ratio')

## 6. Transaction Costs and Leverage Constraints Analysis

### 6.1 Theoretical Framework for Implementation Constraints

1. **Trading Cost Models in Asset Management:**
   1. Definition: The frictional expenses incurred when executing trades in financial markets
   2. Core components: Explicit costs (commissions, fees) and implicit costs (bid-ask spreads, market impact)
   3. Mathematical formulation: $TC = \sum_{t=1}^{T} \sum_{i=1}^{N} |w_{i,t} - w_{i,t-1}^+| \cdot c_i$ where $w_{i,t-1}^+$ is the weight before rebalancing
   4. Impact on performance: Net return = Gross return - Transaction costs

2. **Transaction Cost Components for Different Asset Classes:**
   1. **Factor Implementation Costs:**
      1. Explicit costs: Lower for large-cap factors, higher for small-cap factors
      2. Implicit costs: Significant market impact for momentum due to high turnover
      3. Capacity constraints: Lower liquidity in certain factor legs (e.g., small-cap value)
   2. **Commodity Implementation Costs:**
      1. Futures roll costs: Contango/backwardation effects on roll yield
      2. Exchange fees and broker commissions (relatively standardized)
      3. Bid-ask spreads: Wider for less liquid commodities (e.g., certain agricultural contracts)
      4. Commodity-specific market impact models based on time of day, seasonality

3. **Leverage Constraints and Their Implications:**
   1. Regulatory constraints (e.g., UCITS 2x gross exposure limit for retail funds)
   2. Risk management policy constraints (internal limits based on VaR budgets)
   3. Practical funding constraints (margin requirements for futures)
   4. Behavioral/institutional constraints (client tolerance for leverage)

4. **Academic Sources:**
   1. Frazzini, A., Israel, R., & Moskowitz, T.J. (2018). Trading costs. Working Paper
   2. Novy-Marx, R., & Velikov, M. (2016). A taxonomy of anomalies and their trading costs. Review of Financial Studies, 29(1), 104-147
   3. de Roon, F.A., Nijman, T.E., & Veld, C. (2000). Hedging pressure effects in futures markets. Journal of Finance, 55(3), 1437-1456
   4. DeMiguel, V., Martin-Utrera, A., & Nogales, F.J. (2015). Parameter uncertainty in multiperiod portfolio optimization with transaction costs. Journal of Financial and Quantitative Analysis, 50(6), 1443-1471

### 6.2 Implementation Methodology

1. **Transaction Cost Modeling Framework:**
   1. Define cost parameters specific to each asset class (factors vs. commodities)
   2. Implement proportional cost models (basis points of traded value)
   3. Calculate turnover statistics across rebalancing periods
   4. Measure net-of-cost performance for different strategy variants
   5. Conduct sensitivity analysis across multiple cost scenarios (10, 30, 50 bps)

2. **Optimizing Trading Efficiency:**
   1. Implement rebalancing thresholds to reduce unnecessary trades (e.g., only rebalance when allocation differs by >5%)
   2. Evaluate optimal rebalancing frequency (daily vs. weekly vs. monthly)
   3. Apply portfolio optimization with transaction cost penalty terms
   4. Implement trading rule smoothing to reduce turnover spikes
   5. Analyze trade sizing to manage market impact costs

3. **Leverage Constraint Implementation Methods:**
   1. Hard capping approach (strictly enforce maximum leverage at each rebalancing)
   2. Soft constraint approach (penalty function in optimization framework)
   3. Dynamic leverage adjustment based on market volatility regimes
   4. Scenario analysis of different leverage caps (1x, 2x, 3x, unconstrained)
   5. Evaluate performance impact of leverage constraints across asset classes

4. **Volatility Targeting Strategy Adaptations:**
   1. Introduce trading buffers around target volatility to reduce turnover
   2. Implement partial rebalancing techniques to smooth position changes
   3. Explore adaptive volatility targeting parameters based on cost sensitivity
   4. Analyze trade-off between targeting precision and implementation costs
   5. Test cost-aware volatility scaling algorithms

5. **Implementation Challenges and Solutions:**
   1. Challenge: High turnover during volatility regime shifts
      - Solution: Implement gradual position adjustments over multiple days
   2. Challenge: Commodity-specific roll costs affecting performance
      - Solution: Optimize roll timing and method (Goldman roll, enhanced roll)
   3. Challenge: Heterogeneous trading costs across asset universe
      - Solution: Asset-specific cost models and liquidity-aware position sizing
   4. Challenge: Leverage spikes during sudden volatility drops
      - Solution: Implement volatility forecasting to anticipate regime changes
   5. Challenge: Performance attribution between alpha and implementation effects
      - Solution: Decompose returns into strategy, timing, and cost components

In [ ]:
# Implementation of transaction cost and leverage constraint analysis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def calculate_turnover(positions_df):
    """
    Calculate turnover from position changes
    
    Parameters:
    -----------
    positions_df : pandas.DataFrame
        DataFrame with positions over time
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame with turnover statistics
    """
    # Calculate position changes
    position_changes = positions_df.diff().abs()
    
    # Calculate turnover statistics
    daily_turnover = position_changes.sum(axis=1)
    asset_turnover = position_changes.sum(axis=0)
    
    # Annualize turnover (assuming daily data)
    ann_turnover = daily_turnover.mean() * 252
    
    return {
        'daily_turnover': daily_turnover,
        'asset_turnover': asset_turnover,
        'annualized_turnover': ann_turnover
    }

def apply_transaction_costs(returns_df, positions_df, cost_bps=10):
    """
    Apply transaction costs to returns based on position changes
    
    Parameters:
    -----------
    returns_df : pandas.DataFrame
        DataFrame with strategy returns
    positions_df : pandas.DataFrame
        DataFrame with positions over time
    cost_bps : float or pandas.Series
        Transaction cost in basis points (can be series for asset-specific costs)
        
    Returns:
    --------
    pandas.DataFrame
        Returns after transaction costs
    """
    # Convert cost to decimal
    cost_decimal = cost_bps / 10000
    
    # Calculate position changes
    position_changes = positions_df.diff().abs()
    
    # Calculate costs for each day
    if isinstance(cost_decimal, pd.Series):
        # Asset-specific costs
        costs = position_changes.multiply(cost_decimal, axis=1)
    else:
        # Uniform cost
        costs = position_changes * cost_decimal
    
    # Sum costs across assets for each day
    daily_costs = costs.sum(axis=1)
    
    # Apply costs to returns (align indices)
    aligned_costs = daily_costs.reindex(returns_df.index).fillna(0)
    net_returns = returns_df - aligned_costs
    
    return net_returns

def analyze_cost_sensitivity(returns_df, positions_df, cost_scenarios=[10, 30, 50]):
    """
    Analyze strategy performance under different cost scenarios
    
    Parameters:
    -----------
    returns_df : pandas.DataFrame
        DataFrame with strategy returns
    positions_df : pandas.DataFrame
        DataFrame with positions over time
    cost_scenarios : list
        List of transaction cost scenarios in basis points
        
    Returns:
    --------
    dict
        Dictionary with performance metrics for each cost scenario
    """
    results = {}
    
    # Calculate performance for each cost scenario
    for cost_bps in cost_scenarios:
        # Apply costs
        net_returns = apply_transaction_costs(returns_df, positions_df, cost_bps)
        
        # Calculate performance metrics
        metrics = calculate_performance_metrics(net_returns)
        
        results[f'Cost_{cost_bps}bps'] = metrics
    
    return results

def implement_rebalancing_threshold(positions_df, current_positions, volatility, target_vol, 
                                   threshold_pct=5.0, max_leverage=3.0):
    """
    Implement threshold-based rebalancing to reduce turnover
    
    Parameters:
    -----------
    positions_df : pandas.DataFrame
        DataFrame with historical positions
    current_positions : pandas.Series
        Current positions before rebalancing
    volatility : pandas.Series
        Current volatility estimates
    target_vol : float
        Target volatility level
    threshold_pct : float
        Rebalancing threshold as percentage of position
    max_leverage : float
        Maximum allowed leverage
        
    Returns:
    --------
    pandas.Series
        New positions after threshold-based rebalancing
    """
    # Calculate target positions
    target_positions = (target_vol / volatility).clip(upper=max_leverage)
    
    # Calculate percentage difference between current and target
    pct_diff = abs((target_positions - current_positions) / current_positions) * 100
    
    # Only rebalance positions that exceed threshold
    rebalance_mask = pct_diff > threshold_pct
    
    # Create new positions
    new_positions = current_positions.copy()
    new_positions[rebalance_mask] = target_positions[rebalance_mask]
    
    return new_positions

def plot_cost_sensitivity(cost_results, metric='Sharpe Ratio', figsize=(10, 6)):
    """
    Plot the impact of transaction costs on performance metrics
    
    Parameters:
    -----------
    cost_results : dict
        Dictionary with performance metrics for each cost scenario
    metric : str
        Performance metric to plot
    figsize : tuple
        Figure size
        
    Returns:
    --------
    matplotlib.figure.Figure
        Figure with cost sensitivity plot
    """
    # Extract cost values from keys
    costs = [int(k.split('_')[1].replace('bps', '')) for k in cost_results.keys()]
    
    # Extract metric values for each strategy and cost
    strategies = cost_results[list(cost_results.keys())[0]].columns
    
    # Create DataFrame for plotting
    plot_data = pd.DataFrame(index=costs, columns=strategies)
    
    for strategy in strategies:
        for cost_key, metrics in cost_results.items():
            cost = int(cost_key.split('_')[1].replace('bps', ''))
            plot_data.loc[cost, strategy] = metrics.loc[metric, strategy]
    
    # Sort by cost
    plot_data = plot_data.sort_index()
    
    # Create plot
    fig, ax = plt.subplots(figsize=figsize)
    
    for strategy in strategies:
        ax.plot(plot_data.index, plot_data[strategy], marker='o', label=strategy)
    
    ax.set_title(f'Impact of Transaction Costs on {metric}')
    ax.set_xlabel('Transaction Costs (bps)')
    ax.set_ylabel(metric)
    ax.grid(True, alpha=0.3)
    ax.legend()
    
    plt.tight_layout()
    return fig

# Example usage (would be implemented with actual data)
# turnover_stats = calculate_turnover(strategy_positions['EWMA_Vol'])
# net_returns = apply_transaction_costs(strategy_returns['EWMA_Vol'], strategy_positions['EWMA_Vol'], 30)
# cost_sensitivity = analyze_cost_sensitivity(strategy_returns['EWMA_Vol'], strategy_positions['EWMA_Vol'])
# cost_plot = plot_cost_sensitivity(cost_sensitivity, metric='Sharpe Ratio')